In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from collections import namedtuple
import shortuuid
import pickle
import collections
import json
import sys
import string
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
# initialise spark
spark = SparkSession.builder \
    .master('local') \
    .appName('muthootSample1') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

# load the bloomberg topic taxonomy parquet into a json object
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('data/topic_taxonomy.parquet')
pdf = df.toPandas()
topic_taxonomy = pdf.to_json()
topic_taxonomy = json.loads(topic_taxonomy)


Exception: Java gateway process exited before sending its port number

**TOP-LEVEL TOPIC TAXONOMY:**

| key | desc                      | entity   |
|-----|---------------------------|----------|
| 0   | Business News             | BUSINESS |
| 1   | General News              | GEN      |
| 2   | Legal Affairs, Litigation | LAW      |
| 3   | Miscellaneous             | MISC     |
| 4   | Sports News               | SPORTS   |
| 5   | None                      | None     |
| 6   | Regions                   | WORLD    |

In [4]:
# a function which walks down the tree beneath a node return a list of edges
def walk(node, edges=[], parent='', source=True, level=0):
    desc, code, *children = node 
    uid = shortuuid.uuid()
    edges.append((level, parent, desc, code, uid))
    if len(children) != 0 and children[0] is not None:
        for c in children[0]:
             if c is not None:
                walk(c, edges, parent=code, source=False, level=level+1)
    if source:
        return edges

In [5]:
# create edge list from the topic taxonomy json
edge_list = []
code2desc = {}
for c in ['0', '1', '2', '3', '4', '5', '6']:
    name = topic_taxonomy['Entity'][c]
    desc = topic_taxonomy['Description'][c]
    news = topic_taxonomy['TaxonomyNode'][c]
    edge_list.append((0, 'BBG_NEWS', desc, name, shortuuid.uuid()))
    for i, topics in enumerate(news):
        if topics is not None:
            edge_list.append((1, name, topics[0], topics[1], shortuuid.uuid()))
            edges = []
            edges = walk(topics, edges, level=2)[1:] 
            
            if edges:
                edge_list.extend(edges)

In [6]:
# Print some examples from the edge list
# SCHEMA:
# [level, parent code, child description, child code, parent unique ID]
edge_list[0:7]

[(0, 'BBG_NEWS', 'Business News', 'BUSINESS', 'L5ask7JEn88c7jyPKUBwVo'),
 (1, 'BUSINESS', 'Company News', 'COS', 'LNgmzqFRcjHT6yssCgY4ia'),
 (3, 'COS', 'Analyst Ratings & Commentary', 'ANA', 'yg4YXUBhksUuj5sAGMPCPL'),
 (4, 'ANA', 'Analyst Reiterations', 'ANAAFFR', 'aUT5EWeTmMrL9cn6mEXBDL'),
 (4,
  'ANA',
  'Asia Index Rating Rankings',
  'ANAASARR',
  'omqHcEwjCpLKsxL29EWqJX'),
 (4, 'ANA', 'Analyst Ratings - Buys', 'ANABUY', 'xvFgwE3dbF5kkg6a6RNxDn'),
 (5,
  'ANABUY',
  'Analyst Ratings Events, Announcements - Buys',
  'ANABEVT',
  'PcASE3FLBeFp88Jas9iEoP')]

In [7]:
# Build code2desc {code: desc} (ignoring duplicates)
code2desc = {x[3]:x[2] for x in edge_list}
pickle.dump(code2desc, open('code2desc.pkl', 'wb'))

In [29]:
code2desc['MSCINAMER']

'MSCI North America'

In [24]:
[x for x in edge_list if x[3] == 'NEWSTK']

[(4, 'CACT', 'New Stocks', 'NEWSTK', '4yAyoEu3Zm7Y9Cmf4XUT7o'),
 (5, 'CORPFIN', 'New Stocks', 'NEWSTK', 'arTBBuVvYD7jytpPJcwFcZ'),
 (6, 'PLACE', 'New Stocks', 'NEWSTK', 'WuYKT6fiucpHSH4Cm4iGqE'),
 (4, 'COEVNT', 'New Stocks', 'NEWSTK', 'TiQKsMSNyjNKfyscf56aXK'),
 (4, 'CORPFIN', 'New Stocks', 'NEWSTK', 'jP2JopgyoaPvFjvfviLkJ6'),
 (5, 'PLACE', 'New Stocks', 'NEWSTK', 'P4itpcB3T2RRQxX5PjGZKa'),
 (4, 'STK', 'New Stocks', 'NEWSTK', 'ef5A8MdM4QeJbuygKzD2mf'),
 (4, 'EQUITYKEY', 'New Stocks', 'NEWSTK', 's4UArTCvUZcTjnHVC2JkVK'),
 (4, 'CACT', 'New Stocks', 'NEWSTK', '5PAYdynUnen4xg2Ek49pw9'),
 (5, 'CORPFIN', 'New Stocks', 'NEWSTK', 'teBRAVXRU78sRznvuWGb76'),
 (6, 'PLACE', 'New Stocks', 'NEWSTK', 'skTgcY6Ghs4CShLGeZuDq3'),
 (4, 'COEVNT', 'New Stocks', 'NEWSTK', 'AvxQDU7WCrkkJcGibrNG8n'),
 (4, 'CORPFIN', 'New Stocks', 'NEWSTK', 'cfwAqfkMK4eaHoHXMpDySd'),
 (5, 'PLACE', 'New Stocks', 'NEWSTK', 'zgioiPWMC8nryZv5UbY6k4'),
 (5, 'STK', 'New Stocks', 'NEWSTK', 'ZXVaAneZ5eJuFGS4vjkgzE')]

In [34]:
# Examples which contain oil
oil = [(x[3], x[2]) for x in edge_list if x[2] is not None and 'oil' in x[2].lower() and 'palm' not in x[2].lower()]
print(oil)

[('ANP', 'Brazil Oil Regulator'), ('TNK', 'Oil Tankers'), ('FIELD', 'Oil & Gas Fields'), ('FIELDOUT', 'Oil, Natural Gas Field Outages'), ('OILSANDS', 'Oil Sands'), ('OILSHALE', 'Oil Shale'), ('OILMARKET', 'Oil Markets'), ('CRMKTS', 'Crude Oil Futures'), ('CRRMKT', 'Crude Oil Daily Market Wraps'), ('OFAMKT', 'Oil Products Asia Fuel Oil'), ('OILARB', 'Oil Arbitrage'), ('OILBAR', 'Oil Barometers'), ('OILDAYBK', 'Oil Daybook'), ('ASOILDAYBK', 'Oil Daybook - Asia'), ('EUOILDAYBK', 'Oil Daybook - Europe'), ('USOILDAYBK', 'Oil Daybook - Americas'), ('OILDRV', 'Oil Market Drivers'), ('OILSPILL', 'Oil Spills'), ('OILTOPZ1', 'TOP Oil Zone 1'), ('OILTOPZ2', 'TOP Oil Zone 2'), ('OPMKT', 'Oil Products Markets'), ('OPAMKT', 'Oil Products Asia'), ('OPEMKT', 'European Oil Products Report'), ('OPUMKT', 'Oil Products U.S.'), ('OSP', 'Oil Official Selling Prices'), ('WKOIL', 'BFW Oil Weekly Agenda'), ('OIL', 'Oil'), ('CRUDE', 'Crude Oil'), ('CRMKTS', 'Crude Oil Futures'), ('CRRMKT', 'Crude Oil Daily Mark

In [14]:
# Total topic count
total_topics = len(set([e[1] for e in edge_list] + [e[3] for e in edge_list]))
print(f'total topics: {total_topics}')

total topics: 10147


In [15]:
def cleanup(text):
    text = text.translate(str.maketrans('', '', string.punctuation)).strip()
    text = '_'.join(text.split())
    return text

In [17]:
import csv
writer = csv.writer(open("data/topic_tree.csv", 'w'))
writer.writerow(['parent', 'child'])
for row in edge_list:
    parent = row[1]
    child = row[3]
    if parent is None or child is None:
        continue
    parent = cleanup(parent)
    child = cleanup(child)
    if parent == '' or child == '':
        continue
    writer.writerow([parent, child])
    
    

In [26]:
def create_tree_from_edges(edges):
    tree = {}
    for v in edges:
        tree.setdefault(v[1], []).append(v[3])
    return tree

In [32]:
tree = create_tree_from_edges(edge_list)

In [36]:
tree['OILBRIEF']

['OILBRIEFAM',
 'OILBRIEFAS',
 'OILBRIEFEU',
 'OILBRIEFAM',
 'OILBRIEFAS',
 'OILBRIEFEU',
 'OILBRIEFAM',
 'OILBRIEFAS',
 'OILBRIEFEU']